In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 분양가 파일 로드 인코딩 cp949
df_last = pd.read_csv('./open-data-set/주택도시보증공사_전국 평균 분양가격(2019년 12월).csv', encoding='cp949')
df_last.shape

(4335, 5)

In [ ]:
df_last.head()

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879


In [ ]:
df_last.tail()

,지역명,규모구분,연도,월,분양가격(㎡)
4330,제주,전체,2019,12,3882
4331,제주,전용면적 60㎡이하,2019,12,NaN
4332,제주,전용면적 60㎡초과 85㎡이하,2019,12,3898
4333,제주,전용면적 85㎡초과 102㎡이하,2019,12,NaN
4334,제주,전용면적 102㎡초과,2019,12,3601


In [ ]:
# 전국 평균 분양가격 데이터 들고오기
df_first = pd.read_csv('./open-data-set/전국 평균 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
df_first.shape

(17, 22)

#### 데이터 요약하기

In [ ]:
df_last.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4335 entries, 0 to 4334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   지역명      4335 non-null   object
 1   규모구분     4335 non-null   object
 2   연도       4335 non-null   int64 
 3   월        4335 non-null   int64 
 4   분양가격(㎡)  4058 non-null   object
dtypes: int64(2), object(3)
memory usage: 169.5+ KB


#### 결측치 보기

In [ ]:
# 분양가격(㎡)의 결측치가 277개 인거를 볼 수 있다.

# isnull() 을 통해 결측치를 구합니다.
df_last.isnull().sum()

지역명          0
규모구분         0
연도           0
월            0
분양가격(㎡)    277
dtype: int64

In [ ]:
# isna() 을 통해 결측치를 구합니다.
df_last.isna().sum()

지역명          0
규모구분         0
연도           0
월            0
분양가격(㎡)    277
dtype: int64

#### 데이터 타입 변경

In [ ]:
type(np.nan)

float

In [ ]:
# 분양가격이 object(문자) 타입으로 되어 있어, 데이터 타입을 변경합니다.
# errors='coerce' 옵션으로 강제로 바꿔줍니다.
df_last['분양가격'] = pd.to_numeric(df_last['분양가격(㎡)'], errors='coerce') 
# df_last['분양가격(㎡)'].mean() # 타입 에러 발생
df_last['분양가격'].mean()

np.float64(3238.128632802628)

#### 평당분양가격 구하기
분양가격을 평당가격 기준으로 보기위해 3.3을 곱해서 '평당분양가격' 컬럼을 만들어줍니다.

In [ ]:
df_last['평당분양가격'] = df_last['분양가격'] * 3.3
df_last.head(2)

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6


In [ ]:
df_last.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4335 entries, 0 to 4334
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역명      4335 non-null   object 
 1   규모구분     4335 non-null   object 
 2   연도       4335 non-null   int64  
 3   월        4335 non-null   int64  
 4   분양가격(㎡)  4058 non-null   object 
 5   분양가격     3957 non-null   float64
 6   평당분양가격   3957 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 237.2+ KB


In [ ]:
# 변경 전 컬럼인 분양가격(㎡) 컬럼을 요약합니다. 

df_last['분양가격(㎡)'].describe()

count     4058
unique    1753
top       2221
freq        17
Name: 분양가격(㎡), dtype: object

In [ ]:
# 수치데이터로 변경된 분양가격 컬럼을 요약합니다.
# 평균값과 중앙값의 차이를 봅니다.
# max값이 크기때문에 평균값이 더 크게 나오고 있습니다.

df_last['분양가격'].describe()

count     3957.000000
mean      3238.128633
std       1264.309933
min       1868.000000
25%       2441.000000
50%       2874.000000
75%       3561.000000
max      12728.000000
Name: 분양가격, dtype: float64